In [1]:
import torch
from transformers import PreTrainedTokenizerFast
from transformers import GPT2LMHeadModel

c:\ProgramData\anaconda3\envs\TORCH_NLP38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 스페셜 토큰 정의
Q_TKN = "<usr>"
A_TKN = "<sys>"
BOS = "</s>"
EOS = "</s>"
SENT = '<unused1>'
PAD = "<pad>"
MASK = "<unused0>"
# 사전학습된 토크나이저
TOKENIZER = PreTrainedTokenizerFast.from_pretrained("skt/kogpt2-base-v2", bos_token=BOS, eos_token=EOS, unk_token="<unk>", pad_token=PAD, mask_token=MASK)

C:\Users\kdp\AppData\Roaming\Python\Python38\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kdp\.cache\huggingface\hub\models--skt--kogpt2-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
The tokenizer class you load from this checkpoint is not the same type as the class this function is called

In [3]:
max_input_length = TOKENIZER.model_max_length  # 모델의 최대 입력 길이
vocab_size = TOKENIZER.vocab_size  # 토크나이저의 어휘 크기
print(f"max_input_length: {max_input_length}, vocab_size: {vocab_size}")

max_input_length: 1000000000000000019884624838656, vocab_size: 51200


In [9]:
model_state_dict = torch.load('./model/chatbot_model2.pth', map_location=torch.device('cpu'))
model = GPT2LMHeadModel.from_pretrained("skt/kogpt2-base-v2")
model.load_state_dict(model_state_dict)

<All keys matched successfully>

In [10]:
with torch.no_grad():
    while 1:
        q = input("user > ").strip()
        if q == "quit":
            break
        a = ""
        while 1:
            input_ids = torch.LongTensor(TOKENIZER.encode(Q_TKN + q + SENT + A_TKN + a)).unsqueeze(dim=0)
            print(input_ids)
            pred = model(input_ids)
            pred = pred.logits
            gen = TOKENIZER.convert_ids_to_tokens(torch.argmax(pred, dim=-1).squeeze().numpy().tolist())[-1]
            if gen == EOS:
                break
            a += gen.replace("▁", " ")
        print("Chatbot > {}".format(a.strip()))

tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310,  9971]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310,  9971, 25832]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310,  

We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310,  9971, 25832,
          7621,  9735,   447,   420, 16103, 23892, 24094,  8747,  9241,  7172,
          7182,   739, 29858, 37243, 39077, 21804, 10175, 10433, 13389, 10913,
             3]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310,  9971, 25832,
          7621,  9735,   447,   420, 16103, 23892, 24094,  8747,  9241,  7172,
          7182,   739, 29858, 37243, 39077, 21804, 10175, 10433, 13389, 10913,
             3,     3]])
tensor([[    2, 12396, 33302,  8137,  9068,  7422,  8006, 25856,  9105,  7321,
          8711,  8084,   406,    10,     4,  9563, 10294, 25310,  9971, 25832,
          7621,  9735,   447,   420, 16103, 23892, 24094,  8747,  9241,  7172,
          7182,   739, 29858, 37243, 39077, 21804, 10175, 10433, 13389, 10913,
         

KeyboardInterrupt: 